<a href="https://colab.research.google.com/github/gabrielutomo/Youtube-Music-Hits/blob/main/ANN%20UTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT**

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

In [42]:
file_path = "/content/youtube-top-100-songs-2025 (3).csv"
df = pd.read_csv(file_path)

In [43]:
print("Contoh Data:")
display(df.head())

print("\nInformasi Dataset:")
df.info()


Contoh Data:


,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...,2009014557,Music,YG Entertainment;YG;와이지;K-pop;BLACKPINK;블랙핑크;블...,173,2:53,False,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,ROSÉ,https://www.youtube.com/channel/UCBo1hnzxV9rz3...,19200000
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...","Lady Gaga, Bruno Mars - Die With A Smile (Offi...",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...,1324833300,Music,Lady Gaga;Bruno Mars;Interscope;Pop,252,4:12,False,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
2,Reneé Rapp - Leave Me Alone (Official Music Vi...,Reneé Rapp - Leave Me Alone (Official Music Vi...,"Listen to “BITE ME”, the new album from Reneé ...",2536628,Music,Reneé Rapp;Interscope Records;Pop,160,2:40,False,https://i.ytimg.com/vi/tiPWzFLiz4A/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,Billie Eilish - BIRDS OF A FEATHER (Official M...,Listen to HIT ME HARD AND SOFT: https://billie...,558329099,Music,Billie Eilish;Darkroom/Interscope Records;Alte...,231,3:51,False,https://i.ytimg.com/vi/V9PVRfjEBTI/maxresdefau...,Billie Eilish,https://www.youtube.com/channel/UCDGmojLIoWpXo...,56800000
4,Reneé Rapp - Mad (Official Music Video),Reneé Rapp - Mad (Official Music Video),"Listen to “BITE ME”, the new album from Reneé ...",2113548,Music,Reneé Rapp;Interscope Records;Pop,180,3:00,False,https://i.ytimg.com/vi/xkWQM3flsiY/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000



Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   100 non-null    object
 1   fulltitle               100 non-null    object
 2   description             100 non-null    object
 3   view_count              100 non-null    int64 
 4   categories              100 non-null    object
 5   tags                    85 non-null     object
 6   duration                100 non-null    int64 
 7   duration_string         100 non-null    object
 8   live_status             100 non-null    bool  
 9   thumbnail               100 non-null    object
 10  channel                 100 non-null    object
 11  channel_url             100 non-null    object
 12  channel_follower_count  100 non-null    int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 9.6+ KB


## **preprocessing data baris yang memiliki nilai kosong dihapus agar model tidak error.**



In [44]:
df = df.dropna()
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

print("\nData setelah preprocessing:")
display(df.head())


Data setelah preprocessing:


,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,48,48,54,2009014557,0,47,173,12,False,77,34,13,19200000
1,38,38,38,1324833300,0,21,252,48,False,45,26,2,29600000
2,52,52,32,2536628,0,31,160,5,False,54,36,36,408000
3,6,6,23,558329099,0,4,231,40,False,26,4,16,56800000
4,53,53,33,2113548,0,31,180,16,False,62,36,36,408000


# **Membuat Kolom Target**

In [45]:
target_col = 'view_count'
if df[target_col].dtype != 'int' and df[target_col].dtype != 'float':
    le = LabelEncoder()
    df[target_col] = le.fit_transform(df[target_col])
median_views = df[target_col].median()
df['popular'] = (df[target_col] > median_views).astype(int)

X = df.drop(columns=[target_col, 'popular'])
y = df['popular']

# **Pembagian Data Train dan Test**

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# **menormalkan/standarisasi data numerik.**

In [47]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# **Membuat Arsitektur Model Neural Network**

In [48]:
# 64 = jumlah neuron (unit pemrosesan)
# Relu membantu model belajar pola kompleks dengan cepat
# X jumlah fitur input (berapa kolom yang dipakai untuk prediksi)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # output biner (0/1)
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Proses pelatihan model





In [49]:
# Bagian model.fit().Model belajar selama 50 epoch, dengan batch berisi 8 data. Dari seluruh data training, 20% digunakan sebagai validasi untuk memantau apakah model benar-benar belajar
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=8,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.3987 - loss: 0.7973 - val_accuracy: 0.7857 - val_loss: 0.6034
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5385 - loss: 0.7085 - val_accuracy: 0.8571 - val_loss: 0.5809
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5372 - loss: 0.6892 - val_accuracy: 0.9286 - val_loss: 0.5654
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4736 - loss: 0.6920 - val_accuracy: 1.0000 - val_loss: 0.5505
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6197 - loss: 0.6805 - val_accuracy: 1.0000 - val_loss: 0.5383
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5956 - loss: 0.6575 - val_accuracy: 1.0000 - val_loss: 0.5268
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5483 - loss: 0.6686 - val_accuracy: 1.0000 - val_loss: 0.5197
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6053 - loss: 0.6975 - val_accuracy: 1.0000 - val_loss: 0.5122


# **Hasil Prediksi**

In [51]:
# Membuat prediksi berupa angka probabilitas
# Menghitung akurasi model, yaitu persentase prediksi yang benar.
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
print("\nAkurasi Model: ", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step

Akurasi Model:  0.5882352941176471

Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        10
           1       0.50      0.57      0.53         7

    accuracy                           0.59        17
   macro avg       0.58      0.59      0.58        17
weighted avg       0.60      0.59      0.59        17

